In [3]:
import elasticsearch
import math
import numpy as np
import os
import pytest
import random
import requests
import tarfile
import utils

from collections import Counter
from collections import defaultdict
from elasticsearch import Elasticsearch

In [8]:
es = Elasticsearch()
es.info()

{'name': 'Karl-PC',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'c2RTr28oQWi6_aB1dQbLmA',
 'version': {'number': '7.9.1',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '083627f112ba94dffc1232e8b42b73492789ef91',
  'build_date': '2020-09-01T21:22:21.964974Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [4]:
train = utils.load_dataset('datasets/Wikidata/lcquad2_anstype_wikidata_train.json')
test = utils.load_dataset('datasets/Wikidata/lcquad2_anstype_wikidata_test_gold.json')

In [14]:
FIELDS = ['id', 'question', 'category', 'type']

INDEX_NAME = 'questions'

INDEX_SETTINGS = {
    'mappings': {
            'properties': {
#                 'id': {
#                     'type': 'text',
#                     'term_vector': 'yes',
#                     'analyzer': 'english'
#                 },
                'question': {
                    'type': 'text',
                    'term_vector': 'yes',
                    'analyzer': 'english'
                },
                'category': {
                    'type': 'text',
                    'term_vector': 'yes',
                    'analyzer': 'english'
                },
                'type': {
                    'type': 'keyword',
#                     'term_vector': 'yes',
#                     'analyzer': 'english'
                }
            }
        }
    }

In [24]:
def index_doc(es, doc, doc_id, index='questions'):
    """Indexes the body of a fielded document to be indexed.
    
    Argments:
        es: Elasticsearch object instance.
        doc: Dict with fields as keys and strings as values. Assumes {'title': '...', 'body': '...'}.
        doc_id: Document ID to be used in index. 
        index: Name of index under which documents will be organized.
    """
    es.index(index=index, id=doc_id, body=doc)

In [28]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(INDEX_NAME)
    
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)
count = 0
for x in train:
#     print(x)
#     break
    if x['category'] != 'resource':
        continue
    doc = {
        'question': x['question'],
        'category': x['category'],
        'type': x['type']
    }

    index_doc(es, doc, x['id'], index=INDEX_NAME)
    count += 1
    print('\rIndexed documents: ' + str(count) + '/' + str(len(train)), end='', flush=True)
print('') # Write newline

Indexed documents: 11683/18251


In [32]:
es.get(index=INDEX_NAME, id='6840')

{'_index': 'questions',
 '_type': '_doc',
 '_id': '6840',
 '_version': 1,
 '_seq_no': 10056,
 '_primary_term': 1,
 'found': True,
 '_source': {'question': "What award did Nikolai Andrianov receive for participating in the Gymnastics at the 1976 Summer Olympics - Men's parallel bars?",
  'category': 'resource',
  'type': ['medal', 'Olympic award']}}

In [40]:
# q = 'What periodical literature does Delta Air Lines use as a moutpiece?'
# q = 'What is the country for head of state of Mahmoud Abbas'
q = 'Is cola healthy?'

es.search(index=INDEX_NAME, q=q, _source=True, size=5).get('hits', {}).get('hits', {})

[{'_index': 'questions',
  '_type': '_doc',
  '_id': '30077',
  '_score': 8.535293,
  '_source': {'question': 'Tell me about cola for product or material produced of The Coca-Cola Company?',
   'category': 'resource',
   'type': ['goods', 'physical object', 'class', 'trademark', 'soft drink']}},
 {'_index': 'questions',
  '_type': '_doc',
  '_id': '24488',
  '_score': 7.8734784,
  '_source': {'question': 'which cola starts with the letter p',
   'category': 'resource',
   'type': ['soft drink',
    'trademark',
    'carbonated beverage',
    'non-alcoholic beverage',
    'symbol',
    'class',
    'protected name',
    'intellectual property',
    'sign']}},
 {'_index': 'questions',
  '_type': '_doc',
  '_id': '22455',
  '_score': 7.3887763,
  '_source': {'question': 'Which is the corporation for the products produced by Coca-Cola?',
   'category': 'resource',
   'type': ['juridical person',
    'company',
    'separate legal entity',
    'legal form']}},
 {'_index': 'questions',
  '_t